# PHASE_5_CAPSTONE_PROJECT

# MODELING

### Data source:

The data source consists of over 44 thousand fashion images from Kaggle.

Source link: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset

### Target:

Leverage a dataset comprising more than 24k images across 10 distinct fashion product types, including t-shirts, shirts, casual shoes, watches, sports shoes, kurtas, handbags, heels, sunglasses, and wallets, for image labeling in customer search.

In [1]:
import pandas as pd
import numpy as np
import csv
import json
import glob
import os
import shutil
import random
import pickle
import h5py
import hdf5storage
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from IPython.display import display
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras import layers
from keras.models import Sequential
from keras.layers import Reshape

from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array
from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import Dropout
from keras.models import load_model


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

# a new layer that rescales/normalizes the activations after each layer.
# has a weak regularization effect. also allows 

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers, regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Resizing, RandomFlip, RandomRotation
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

%matplotlib inline


In [2]:
# !pip install tensorflow

In [3]:
# Directory path
train_data_dir_sample = '/Users/seanluong/Downloads/TransferX/Train_Test_Validation/Train'
val_data_dir_sample = '/Users/seanluong/Downloads/TransferX/Train_Test_Validation/Validation'
test_data_dir_sample = '/Users/seanluong/Downloads/TransferX/Train_Test_Validation/Test'




# Define the image size and number of classes
image_size_224 = (224, 224)
num_classes = 10 


# specialized mobilenetv2 preprocesser
mobilenetv2_processor = tf.keras.applications.mobilenet_v2.preprocess_input

# Create the train_datagen with vgg_processor
train_datagen_mobilenetv2 = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.3, 
                                   zoom_range=0.1, 
                                   horizontal_flip=False,
                                   preprocessing_function=mobilenetv2_processor)

# Create the val_datagen with vgg_processor
val_datagen_mobilenetv2 = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function=mobilenetv2_processor)

# Create the test_datagen with vgg_processor
test_datagen_mobilenetv2 = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=mobilenetv2_processor)

# ...

# Flow data from the directory using the data generators
train_generator_mobilenetv2 = train_datagen_mobilenetv2.flow_from_directory(directory=train_data_dir_sample,
                                                    target_size=image_size_224,
                                                    class_mode='categorical',
                                                    batch_size=64,
                                                    shuffle=True,
                                                    classes=[str(i) for i in range(num_classes)])

val_generator_mobilenetv2 = val_datagen_mobilenetv2.flow_from_directory(directory=val_data_dir_sample,
                                                target_size=image_size_224,
                                                class_mode='categorical',
                                                batch_size=64,
                                                shuffle=True,
                                                classes=[str(i) for i in range(num_classes)])

test_generator_mobilenetv2 = test_datagen_mobilenetv2.flow_from_directory(directory=test_data_dir_sample,
                                                  target_size=image_size_224,
                                                  class_mode='categorical',
                                                  batch_size=64,
                                                  shuffle=False,
                                                  classes=[str(i) for i in range(num_classes)])


Found 14782 images belonging to 10 classes.
Found 4935 images belonging to 10 classes.
Found 4926 images belonging to 10 classes.


### MobileNetV2 Model

In [4]:
base_mobilenetv2_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in base_mobilenetv2_model.layers:
    layer.trainable = False


In [5]:
len(base_mobilenetv2_model.layers)

154

In [6]:
base_mobilenetv2_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, None, None, 32)       128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, None, None, 32)       0         ['bn_Conv1[

In [7]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [8]:
mobilenetv2_model = custom_classifier(base_mobilenetv2_model, num_classes)

In [9]:
mobilenetv2_model.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [33]:
history_mobilenetv2_model = mobilenetv2_model.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=50,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/50
231/231 [==============================] - 111s 481ms/step - loss: 1.9862 - accuracy: 0.3153 - auc: 0.7379 - precision: 0.5106 - recall: 0.0049 - val_loss: 1.7678 - val_accuracy: 0.4239 - val_auc: 0.8353 - val_precision: 0.3208 - val_recall: 0.0034
Epoch 2/50
231/231 [==============================] - 110s 473ms/step - loss: 1.8599 - accuracy: 0.3437 - auc: 0.7888 - precision: 0.5323 - recall: 0.0251 - val_loss: 1.5701 - val_accuracy: 0.4006 - val_auc: 0.8713 - val_precision: 0.4780 - val_recall: 0.1980
Epoch 3/50
231/231 [==============================] - 110s 476ms/step - loss: 1.7418 - accuracy: 0.3722 - auc: 0.8251 - precision: 0.5511 - recall: 0.0423 - val_loss: 1.4762 - val_accuracy: 0.4328 - val_auc: 0.8844 - val_precision: 0.6811 - val_recall: 0.1117
Epoch 4/50
231/231 [==============================] - 117s 507ms/step - loss: 1.6525 - accuracy: 0.3915 - auc: 0.8459 - precision: 0.5992 - recall: 0.0670 - val_loss: 1.4076 - val_accuracy: 0.4324 - val_auc: 0.8926 - val

#### Save model & history

In [92]:
mobilenetv2_model.save('mobilenetv2_base_model_50epoch.h5')

In [93]:
hdf5storage.savemat("history_mobilenetv2_base_model_epoch.h5", history_mobilenetv2_model.history)

#### Evaluate the model

In [94]:
base_mobilenetv2_model_loaded = load_model('mobilenetv2_base_model_50epoch.h5')

In [ ]:
base_mobilenetv2_model_loaded.evaluate(train_generator_mobilenetv2)

 69/231 [=======>......................] - ETA: 58s - loss: 1.1541 - accuracy: 0.5620 - auc: 0.9263 - precision: 0.7516 - recall: 0.3419

In [ ]:
base_mobilenetv2_model_loaded.evaluate(test_generator_mobilenetv2)

In [ ]:
history_base_mobilenetv2_model_loaded = hdf5storage.loadmat("history_mobilenetv2_base_model_epoch.h5")

In [ ]:
print(history_base_mobilenetv2_model_loaded.keys())

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))

base_mobilenev2_loss_values = history_base_mobilenetv2_model_loaded['loss']
base_mobilenev2__val_loss_values = history_base_mobilenetv2_model_loaded['val_loss']


epochs = range(1,len(base_mobilenev2_loss_values)+1)

ax.plot(epochs,base_mobilenev2_loss_values,label='Train loss vs epochs')
ax.plot(epochs,base_mobilenev2__val_loss_values,label='Validation loss vs epochs')

ax.set_title('Train & validation loss')
ax.legend()
ax.set_xlabel('Epochs')
plt.grid()
ax.set_ylabel('Loss');

In [43]:
mobilenetv2_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, None, None, 32)       864       ['input_2[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, None, None, 32)       128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, None, None, 32)       0         ['bn_Conv1[0][0]']        

In [44]:
len(mobilenetv2_model.layers)

157

### Model tuning

Unfreeze last 2 layers

In [11]:
mobilenetv2_model_tune1 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_model_tune1.layers[-2:]:
    layer.trainable = True

In [12]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [13]:
mobilenetv2_model_tune1 = custom_classifier(mobilenetv2_model_tune1, num_classes)

In [14]:
mobilenetv2_model_tune1.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [67]:
history_mobilenetv2_model_tune1 = mobilenetv2_model_tune1.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=10,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 477s 2s/step - loss: 0.2882 - accuracy: 0.8994 - auc: 0.9936 - precision: 0.9164 - recall: 0.8847 - val_loss: 16.3582 - val_accuracy: 0.1305 - val_auc: 0.5169 - val_precision: 0.1305 - val_recall: 0.1305
Epoch 2/5
231/231 [==============================] - 474s 2s/step - loss: 0.1806 - accuracy: 0.9369 - auc: 0.9969 - precision: 0.9427 - recall: 0.9311 - val_loss: 12.0533 - val_accuracy: 0.1305 - val_auc: 0.5232 - val_precision: 0.1305 - val_recall: 0.1305
Epoch 3/5
231/231 [==============================] - 474s 2s/step - loss: 0.1460 - accuracy: 0.9472 - auc: 0.9977 - precision: 0.9519 - recall: 0.9428 - val_loss: 5.3613 - val_accuracy: 0.1305 - val_auc: 0.5813 - val_precision: 0.1305 - val_recall: 0.1305
Epoch 4/5
231/231 [==============================] - 475s 2s/step - loss: 0.1353 - accuracy: 0.9512 - auc: 0.9980 - precision: 0.9548 - recall: 0.9480 - val_loss: 11.1334 - val_accuracy: 0.0713 - val_auc: 0.4670 - val_precision: 0

### Model tunning 2

In [15]:
mobilenetv2_model_tune2 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_model_tune2.layers[-2:]:
    layer.trainable = True

In [16]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu',kernel_regularizer=l2(0.001))(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [17]:
mobilenetv2_model_tune2 = custom_classifier(mobilenetv2_model_tune2, num_classes)

In [18]:
mobilenetv2_model_tune2.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [19]:
history_mobilenetv2_model_tune2 = mobilenetv2_model_tune2.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 500s 2s/step - loss: 0.4867 - accuracy: 0.8918 - auc: 0.9933 - precision: 0.9095 - recall: 0.8778 - val_loss: 20.4762 - val_accuracy: 0.0381 - val_auc: 0.4656 - val_precision: 0.0381 - val_recall: 0.0381
Epoch 2/5
231/231 [==============================] - 498s 2s/step - loss: 0.2764 - accuracy: 0.9403 - auc: 0.9970 - precision: 0.9444 - recall: 0.9343 - val_loss: 18.2091 - val_accuracy: 0.1305 - val_auc: 0.5169 - val_precision: 0.1305 - val_recall: 0.1305
Epoch 3/5
231/231 [==============================] - 481s 2s/step - loss: 0.2105 - accuracy: 0.9470 - auc: 0.9975 - precision: 0.9505 - recall: 0.9428 - val_loss: 20.3655 - val_accuracy: 0.0381 - val_auc: 0.4656 - val_precision: 0.0381 - val_recall: 0.0381
Epoch 4/5
231/231 [==============================] - 480s 2s/step - loss: 0.1684 - accuracy: 0.9531 - auc: 0.9979 - precision: 0.9551 - recall: 0.9502 - val_loss: 17.4594 - val_accuracy: 0.0381 - val_auc: 0.4659 - val_precision: 

### Model tuning 3

In [38]:
mobilenetv2_tune3 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune3.layers:
    layer.trainable = False

In [39]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [40]:
mobilenetv2_model_tune3 = custom_classifier(mobilenetv2_tune3, num_classes)

In [41]:
mobilenetv2_model_tune3.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [42]:
history_mobilenetv2_model_tune3 = mobilenetv2_model_tune3.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 114s 489ms/step - loss: 2.0910 - accuracy: 0.3031 - auc: 0.7216 - precision: 0.3958 - recall: 0.0202 - val_loss: 1.5112 - val_accuracy: 0.4057 - val_auc: 0.8730 - val_precision: 0.6720 - val_recall: 0.0170
Epoch 2/5
231/231 [==============================] - 113s 488ms/step - loss: 1.5852 - accuracy: 0.3937 - auc: 0.8538 - precision: 0.5486 - recall: 0.1015 - val_loss: 1.3517 - val_accuracy: 0.4513 - val_auc: 0.8955 - val_precision: 0.6242 - val_recall: 0.1599
Epoch 3/5
231/231 [==============================] - 113s 489ms/step - loss: 1.4760 - accuracy: 0.4225 - auc: 0.8749 - precision: 0.5898 - recall: 0.1541 - val_loss: 1.1854 - val_accuracy: 0.5473 - val_auc: 0.9259 - val_precision: 0.6878 - val_recall: 0.2746
Epoch 4/5
231/231 [==============================] - 113s 490ms/step - loss: 1.3945 - accuracy: 0.4568 - auc: 0.8889 - precision: 0.6110 - recall: 0.2023 - val_loss: 1.2283 - val_accuracy: 0.4503 - val_auc: 0.9121 - val_pre

### Model tuning 4

In [48]:
mobilenetv2_tune4 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune4.layers:
    layer.trainable = False

In [49]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [50]:
mobilenetv2_model_tune4 = custom_classifier(mobilenetv2_tune4, num_classes)

In [51]:
mobilenetv2_model_tune4.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [52]:
history_mobilenetv2_model_tune4 = mobilenetv2_model_tune4.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 120s 515ms/step - loss: 1.9754 - accuracy: 0.3189 - auc: 0.7496 - precision: 0.4807 - recall: 0.0378 - val_loss: 1.5274 - val_accuracy: 0.3157 - val_auc: 0.8676 - val_precision: 0.4410 - val_recall: 0.0667
Epoch 2/5
231/231 [==============================] - 117s 506ms/step - loss: 1.5556 - accuracy: 0.3930 - auc: 0.8588 - precision: 0.5551 - recall: 0.1329 - val_loss: 1.2414 - val_accuracy: 0.4652 - val_auc: 0.9123 - val_precision: 0.6183 - val_recall: 0.2924
Epoch 3/5
231/231 [==============================] - 119s 515ms/step - loss: 1.4410 - accuracy: 0.4382 - auc: 0.8801 - precision: 0.6074 - recall: 0.1929 - val_loss: 1.1410 - val_accuracy: 0.5224 - val_auc: 0.9263 - val_precision: 0.6674 - val_recall: 0.2561
Epoch 4/5
231/231 [==============================] - 114s 491ms/step - loss: 1.3834 - accuracy: 0.4586 - auc: 0.8901 - precision: 0.6323 - recall: 0.2281 - val_loss: 1.3154 - val_accuracy: 0.4458 - val_auc: 0.9005 - val_pre

### Model tuning 5

In [58]:
mobilenetv2_tune5 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune5.layers:
    layer.trainable = False

In [59]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [60]:
mobilenetv2_model_tune5 = custom_classifier(mobilenetv2_tune5, num_classes)

In [61]:
mobilenetv2_model_tune5.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [62]:
history_mobilenetv2_model_tune5 = mobilenetv2_model_tune5.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 118s 506ms/step - loss: nan - accuracy: 0.2870 - auc: 0.5000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2863 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
231/231 [==============================] - 113s 486ms/step - loss: nan - accuracy: 0.2870 - auc: 0.5000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2863 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
231/231 [==============================] - 112s 483ms/step - loss: nan - accuracy: 0.2870 - auc: 0.5000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: nan - val_accuracy: 0.2863 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/5
231/231 [==============================] - 112s 482ms/step - loss: nan - accuracy: 0.2870 - auc: 0.5000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: nan - val_accuracy: 0.

### Model tuning 6

In [63]:
mobilenetv2_tune6 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune6.layers:
    layer.trainable = False

In [64]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [65]:
mobilenetv2_model_tune6 = custom_classifier(mobilenetv2_tune6, num_classes)

In [66]:
mobilenetv2_model_tune6.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [67]:
history_mobilenetv2_model_tune6 = mobilenetv2_model_tune6.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 120s 515ms/step - loss: 2.0719 - accuracy: 0.3100 - auc: 0.7410 - precision: 0.4083 - recall: 0.0265 - val_loss: 1.4507 - val_accuracy: 0.3974 - val_auc: 0.8799 - val_precision: 0.6274 - val_recall: 0.0464
Epoch 2/5
231/231 [==============================] - 119s 514ms/step - loss: 1.5767 - accuracy: 0.3891 - auc: 0.8540 - precision: 0.5446 - recall: 0.1197 - val_loss: 1.3005 - val_accuracy: 0.4480 - val_auc: 0.9020 - val_precision: 0.5685 - val_recall: 0.3001
Epoch 3/5
231/231 [==============================] - 118s 512ms/step - loss: 1.4346 - accuracy: 0.4349 - auc: 0.8816 - precision: 0.6179 - recall: 0.1871 - val_loss: 1.2019 - val_accuracy: 0.4960 - val_auc: 0.9199 - val_precision: 0.6218 - val_recall: 0.1949
Epoch 4/5
231/231 [==============================] - 119s 513ms/step - loss: 1.3640 - accuracy: 0.4638 - auc: 0.8932 - precision: 0.6361 - recall: 0.2377 - val_loss: 1.1217 - val_accuracy: 0.5025 - val_auc: 0.9278 - val_pre

### Tuning 7 

In [74]:
mobilenetv2_tune7 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune7.layers:
    layer.trainable = False

In [75]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [76]:
mobilenetv2_model_tune7 = custom_classifier(mobilenetv2_tune7, num_classes)

In [77]:
mobilenetv2_model_tune7.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [78]:
history_mobilenetv2_model_tune7 = mobilenetv2_model_tune7.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=50,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/50
231/231 [==============================] - 120s 517ms/step - loss: 2.0369 - accuracy: 0.3118 - auc: 0.7396 - precision: 0.4191 - recall: 0.0261 - val_loss: 1.4364 - val_accuracy: 0.4012 - val_auc: 0.8824 - val_precision: 0.4832 - val_recall: 0.2065
Epoch 2/50
231/231 [==============================] - 120s 517ms/step - loss: 1.5834 - accuracy: 0.3899 - auc: 0.8527 - precision: 0.5264 - recall: 0.0945 - val_loss: 1.3825 - val_accuracy: 0.3611 - val_auc: 0.8875 - val_precision: 0.5113 - val_recall: 0.1289
Epoch 3/50
231/231 [==============================] - 120s 518ms/step - loss: 1.4867 - accuracy: 0.4205 - auc: 0.8718 - precision: 0.5737 - recall: 0.1529 - val_loss: 1.3114 - val_accuracy: 0.4774 - val_auc: 0.9038 - val_precision: 0.7113 - val_recall: 0.2097
Epoch 4/50
231/231 [==============================] - 120s 518ms/step - loss: 1.3917 - accuracy: 0.4543 - auc: 0.8890 - precision: 0.6159 - recall: 0.2168 - val_loss: 1.1603 - val_accuracy: 0.4888 - val_auc: 0.9235 - val

### Model tuning 8

In [88]:
mobilenetv2_tune8 = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

for layer in mobilenetv2_tune8.layers[-2:]:
    layer.trainable = True

In [89]:
def custom_classifier(base_model, num_classes):
    x = base_model.output
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [90]:
mobilenetv2_model_tune8 = custom_classifier(mobilenetv2_tune8, num_classes)

In [91]:
mobilenetv2_model_tune8.compile(loss='categorical_crossentropy',
                             optimizer='Adam',
                             metrics=['accuracy', 'AUC', 'Precision', 'Recall'])

In [92]:
history_mobilenetv2_model_tune8 = mobilenetv2_model_tune8.fit(train_generator_mobilenetv2,
                                                  steps_per_epoch=len(train_generator_mobilenetv2),
                                                  epochs=5,
                                                  validation_data=val_generator_mobilenetv2,
                                                  validation_steps=len(val_generator_mobilenetv2))

Epoch 1/5
231/231 [==============================] - 510s 2s/step - loss: 0.5045 - accuracy: 0.8311 - auc: 0.9834 - precision: 0.8754 - recall: 0.7943 - val_loss: 9.9480 - val_accuracy: 0.2863 - val_auc: 0.6159 - val_precision: 0.2863 - val_recall: 0.2863
Epoch 2/5
231/231 [==============================] - 511s 2s/step - loss: 0.2365 - accuracy: 0.9233 - auc: 0.9947 - precision: 0.9323 - recall: 0.9134 - val_loss: 12.1034 - val_accuracy: 0.2863 - val_auc: 0.6008 - val_precision: 0.2863 - val_recall: 0.2863
Epoch 3/5
231/231 [==============================] - 514s 2s/step - loss: 0.1971 - accuracy: 0.9349 - auc: 0.9960 - precision: 0.9415 - recall: 0.9284 - val_loss: 10.0715 - val_accuracy: 0.0748 - val_auc: 0.5829 - val_precision: 0.0748 - val_recall: 0.0748
Epoch 4/5
231/231 [==============================] - 511s 2s/step - loss: 0.1857 - accuracy: 0.9374 - auc: 0.9964 - precision: 0.9434 - recall: 0.9318 - val_loss: 11.0487 - val_accuracy: 0.0381 - val_auc: 0.4646 - val_precision: 0